In [3]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# Prepare Dataset

In [4]:
# Import and clean dataset

df = pd.read_csv(Path("../../Resources/relabeled_data.csv"))
df.dropna(inplace=True)

# Set X and y

X = df.drop(columns='status')
y = df['status']

# Scale X

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Resampling
sampler = SMOTE(random_state=1)
X_resample, y_resample = sampler.fit_resample(X_scaled, y)
display(X_resample.shape)
display(y_resample.shape)

(146924, 18)

(146924,)

# Set instances

In [1]:
### SKLearn MLP Classifier ###
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier()
# first hidden layer = 100
# activation = 'relu'
# optimizer = 'adam'
# epochs = 200

### SKLearn SVC: kernel = rbf ###
from sklearn.svm import SVC
svc_model = SVC(kernel='rbf')


### Tensorflow Keras Sequential ###
nn_model_1 = tf.keras.models.load_model(Path("../../Resources/neural_network_models/four_layers.h5"))
# first hidden layer = 16
# second hidden layer = 8
# third hidden layer = 4
# fourth hidden layer = 2
# hidden layer activation = 'selu'
# output layer activation = 'softplus'
# optimizer = 'nadam'
# epochs = 100

nn_model_2 = tf.keras.models.load_model(Path("../../Resources/neural_network_models/96nodes_4layers_32epochs.h5"))
# first hidden layer = 96
# second hidden layer = 48
# third hidden layer = 24
# fourth hidden layer = 12
# hidden layer activation = 'selu'
# output layer activation = 'softplus'
# optimizer = 'nadam'
# epochs = 32

### ImbLearn Easy Ensemble Classifier ###
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier()

### ImbLearn RUS Boost Classifier ###
from imblearn.ensemble import RUSBoostClassifier
rbc_model = RUSBoostClassifier()

### ImbLearn Balanced Baggig Classifier ###
from imblearn.ensemble import BalancedBaggingClassifier
bbc_model = BalancedBaggingClassifier()

NameError: name 'tf' is not defined

# kFold cross validation

In [7]:
# Create ordered dictionary that stores model instances
from collections import OrderedDict

# X_resample, y_resample
sklearn_dictionary = OrderedDict()
sklearn_dictionary['MLPClassifier'] = mlp_model
sklearn_dictionary['SVC kernel rbf'] = svc_model

# X_resample, y_resample
nn_dictionary = OrderedDict()
nn_dictionary['Neural Network 1'] = nn_model_1
nn_dictionary['Neural Network 2'] = nn_model_2

# X_scaled, y
imblearn_dictionary = OrderedDict()
imblearn_dictionary['Easy Ensemble Classifier'] = eec_model
imblearn_dictionary['RUS Boost Classifier'] = rbc_model
imblearn_dictionary['Balanced Bagging Classifier'] = bbc_model

### Define function to perform KFolds

In [61]:
def apply_prediction(model, X_train, X_test, y_train):

    model.fit(X_train, y_train)
    prediction_array = model.predict(X_test)
    
    return prediction_array

In [89]:
def perform_kfold_split(model, X, y):

    kfold = KFold(n_splits=5)

    for train_index, test_index in kfold.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        prediction =  apply_prediction(model, X_train, X_test, y_train)
        return y_test, prediction


In [90]:
def combine_kfold_predictions(models_dict, X, y):

    counter = 0
    prediction_dict = {}

    for key, value in models_dict.items():

        kfold = KFold(n_splits=5)

        for train_index, test_index in kfold.split(X):
            
            counter += 1

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            prediction =  apply_prediction(value, X_train, X_test, y_train)
            
            df = pd.concat([pd.Series(y_test), pd.Series(prediction)], axis=1)
            df.columns = ['test', 'pred']

            prediction_dict[f"{key}_iter{counter}"] = df

        return prediction_dict

In [91]:
# Fast performing machine learning dict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression()
rf_classifier = RandomForestClassifier(n_estimators = 5, criterion = 'entropy')

fast_performing_ml_dict = OrderedDict()
fast_performing_ml_dict['Logistic Regression'] = lr
fast_performing_ml_dict['Random Forest Classifier'] = rf_classifier

In [92]:
a = combine_kfold_predictions(fast_performing_ml_dict, X_resample, y_resample)

In [93]:
a

{'Logistic Regression_iter1':        test  pred
 0         0     1
 1         0     1
 2         0     1
 3         0     1
 4         0     1
 ...     ...   ...
 29380     0     1
 29381     0     1
 29382     0     1
 29383     0     1
 29384     0     1
 
 [29385 rows x 2 columns],
 'Logistic Regression_iter2':        test  pred
 29385   0.0   NaN
 29386   0.0   NaN
 29387   0.0   NaN
 29388   0.0   NaN
 29389   0.0   NaN
 ...     ...   ...
 29380   NaN   1.0
 29381   NaN   1.0
 29382   NaN   1.0
 29383   NaN   0.0
 29384   NaN   0.0
 
 [58770 rows x 2 columns],
 'Logistic Regression_iter3':        test  pred
 58770   0.0   NaN
 58771   0.0   NaN
 58772   0.0   NaN
 58773   0.0   NaN
 58774   0.0   NaN
 ...     ...   ...
 29380   NaN   1.0
 29381   NaN   1.0
 29382   NaN   0.0
 29383   NaN   1.0
 29384   NaN   1.0
 
 [58770 rows x 2 columns],
 'Logistic Regression_iter4':        test  pred
 88155   1.0   NaN
 88156   1.0   NaN
 88157   1.0   NaN
 88158   1.0   NaN
 88159   1.0   NaN